In [2]:
!git clone https://github.com/qq1263632494/MyWizardOfWikipedia.git
!cp -r MyWizardOfWikipedia/parlai .
!cp MyWizardOfWikipedia/AgentModel.py .
!cp MyWizardOfWikipedia/CommonUtils.py .
!cp MyWizardOfWikipedia/DataUtils.py .
!cp MyWizardOfWikipedia/FeatureUtils.py .
!cp MyWizardOfWikipedia/FileUtils.py .
!cp MyWizardOfWikipedia/TrainUtils.py .
!cp MyWizardOfWikipedia/ModelUtils.py .

fatal: destination path 'MyWizardOfWikipedia' already exists and is not an empty directory.


In [3]:
!pip install subword_nmt transformers

     |████████████████████████████████| 675kB 3.9MB/s 
     |████████████████████████████████| 890kB 14.8MB/s 
     |████████████████████████████████| 1.1MB 15.9MB/s 
     |████████████████████████████████| 3.8MB 23.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=177a8eeac921b798a00c75879a0dc6023e35f4111bdad1093f306dba6c945b03
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from AgentModel import ModelOption, TransformerMemNetAgent, TransformerMemNet
from parlai.core.dict import DictionaryAgent
from parlai.core.opt import Opt
from DataUtils import compile_all_dialogs, compile_one_dialog, display_example
from FeatureUtils import Tokenizer
import pickle
import torch
from FileUtils import read_json
from torch.utils.data import DataLoader
from TrainUtils import ARG, final_train_function, WizardOfWikipediaDataset, no_schedule_train
from CommonUtils import tqdm

In [0]:
with open('/content/drive/My Drive/WizardOfWikipedia/dictionary.pickle', 'rb') as fi:
    dictionary = pickle.load(fi)

In [0]:
embedding = torch.load('/content/drive/My Drive/WizardOfWikipedia/fasttext-subword-300-embedding.pytorch')

In [0]:
tokenizer = Tokenizer(dictionary=dictionary, pad_to_max=True, max_length=64)

In [5]:
DATA_PATH = '/content/drive/My Drive/wizard_of_wikipedia/train.json'
DATA = read_json(DATA_PATH)
examples = compile_all_dialogs(DATA)
len(examples)

83247

In [6]:
real_examples = []
for example in examples:
    if example.choosen_index >= 0 and len(example.knowledge_pool) >= 32:
        real_examples.append(example)
len(real_examples)

71620

In [17]:
all_data = []
for example in tqdm(real_examples[6400:51200]):
    all_data.append(tokenizer.tokenize_example_as_train_data(example))

 :44704/44800 还需0.00小时；速度：448.35 iter/s

In [0]:
dataset = WizardOfWikipediaDataset(all_data)

In [0]:
agent = TransformerMemNetAgent(opt=ModelOption(), 
                               tokenizer=tokenizer, 
                               embedding=embedding)

In [0]:
arg = ARG(num_train_epochs=10, 
          learning_rate=5e-4, 
          device="cuda:0", 
          train_batch_size=32, 
          print_every=200, 
          warmup_proportion=0.4, 
          save_dir='/content/drive/My Drive/WOWTMP', 
          warmup_method='linear')

In [24]:
arg.num_train_epochs

10

In [25]:
#no_schedule_train(args=arg, train_dataset=dataset, agent=agent)
final_train_function(args=arg, train_dataset=dataset, agent=agent)

Epoch :1/10 还需0.00秒
Iteration :199/1400 还需0.15小时；速度：2.20 iter/s8.956305503845215
Iteration :399/1400 还需0.13小时；速度：2.17 iter/s7.278573989868164
Iteration :599/1400 还需0.10小时；速度：2.20 iter/s6.381223678588867
Iteration :799/1400 还需0.08小时；速度：2.21 iter/s6.333219051361084
Iteration :999/1400 还需0.05小时；速度：2.20 iter/s6.3889946937561035
Iteration :1199/1400 还需0.03小时；速度：2.16 iter/s6.344010353088379
Iteration :1399/1400 还需0.00小时；速度：2.20 iter/s6.068996906280518
Iteration :1400/1400 还需0.00小时；速度：2.20 iter/s
7.109086114338466
当前模型保存在：/content/drive/My Drive/WOWTMP/EP1.pth
Epoch :2/10 还需5755.96秒
Iteration :199/1400 还需0.15小时；速度：2.18 iter/s6.212854385375977
Iteration :399/1400 还需0.13小时；速度：2.18 iter/s5.786304473876953
Iteration :599/1400 还需0.10小时；速度：2.19 iter/s6.163844585418701
Iteration :799/1400 还需0.08小时；速度：2.21 iter/s5.980534076690674
Iteration :999/1400 还需0.05小时；速度：2.20 iter/s5.8456525802612305
Iteration :1199/1400 还需0.03小时；速度：2.20 iter/s5.766524791717529
Iteration :1399/1400 还需0.00小时；速度：2.20 iter/s5.983

(14001, 5.326606239423472)

In [39]:
!nvidia-smi

Sun Jun  7 10:51:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    42W / 250W |  16277MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  